In [9]:
from synthcity.metrics.eval import Metrics
import pandas as pd
def synthcity_evaluate(real_path, syn_path, result_dir, result_name, real_size, syn_size): 
    real_data = pd.read_csv(real_path)
    syn_data = pd.read_csv(syn_path)
    
    if not real_size:
        real_size = len(real_data)
    if not syn_size:
        syn_size = len(syn_data)
    
    if real_size < len(real_data):
        real_data = real_data.sample(n=real_size, random_state=42)  # Adjust random_state as desired for reproducibility
    if syn_size < len(syn_data):
        syn_data = syn_data.sample(n=syn_size, random_state=42)
    
    synthcity_result = Metrics.evaluate(real_data, syn_data)
    synthcity_result.to_csv(result_name + ".csv")
    return synthcity_result
    
def compare_results(dataset1_result_path, dataset2_result_path):
    dataset1_result = pd.read_csv(dataset1_result_path)[['Unnamed: 0', 'min', 'direction']]
    dataset2_result = pd.read_csv(dataset2_result_path)[['Unnamed: 0', 'min', 'direction']]
    
    # Merge dataset1_result and dataset2_result based on the 'Unnamed: 0' column
    merged_result = pd.merge(dataset1_result, dataset2_result[['Unnamed: 0', 'min']], on='Unnamed: 0', suffixes=('_dataset1', '_dataset2'))
    
    # Reorder columns in merged_result
    # Get a list of columns excluding 'direction'
    columns_reordered = [col for col in merged_result.columns if col != 'direction']
    
    # Append 'direction' column to the end of the list
    columns_reordered.append('direction')
    
    # Reassign the DataFrame with reordered columns
    merged_result = merged_result[columns_reordered]
    return merged_result

In [8]:
compare_results("/Users/minhkau/Documents/TUDelft/Year 3/RP/Code/tabular-gpt/evaluation/synthcity_result_watermarked_100.csv",
                "/Users/minhkau/Documents/TUDelft/Year 3/RP/Code/tabular-gpt/evaluation/synthcity_result_20k.csv")

,Unnamed: 0,min_dataset1,min_dataset2,direction
0,sanity.data_mismatch.score,0.062500,0.125000,minimize
1,sanity.common_rows_proportion.score,0.000000,0.000000,minimize
2,sanity.nearest_syn_neighbor_distance.mean,0.183566,0.000829,minimize
3,sanity.close_values_probability.score,0.650000,0.999450,maximize
4,sanity.distant_values_probability.score,0.020000,0.000050,minimize
5,stats.jensenshannon_dist.marginal,0.057305,0.017926,minimize
6,stats.chi_squared_test.marginal,0.399312,0.792003,maximize
7,stats.inv_kl_divergence.marginal,0.505804,0.850483,maximize
8,stats.ks_test.marginal,0.544667,0.867063,maximize
9,stats.max_mean_discrepancy.joint,0.020075,0.000104,minimize
